In [54]:
%pip install langchain langchain-community langchain-ollama faiss-cpu tiktoken

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [60]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# Load data from a text file
loader = TextLoader("/home/azureuser/User_Sasi/ai_chatbot/Medical Rerort - Michael Johnson - Panic Attack Disorder.txt")  # Replace with your knowledge base
documents = loader.load()
print("📄 Loaded document from knowledge_base.txt")

# Split data into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(f"🧩 Document split into {len(docs)} chunks.")

# Create a vector store using Ollama's embeddings and ChromaDB
embeddings = OllamaEmbeddings(model="llama3")
vector_store = FAISS.from_documents(docs, embeddings)

print("✅ Embeddings stored in ChromaDB.")

# Initialize a retriever for querying the vector store
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
print("🔍 Retriever initialized.")

📄 Loaded document from knowledge_base.txt
🧩 Document split into 5 chunks.
✅ Embeddings stored in ChromaDB.
🔍 Retriever initialized.


In [61]:
from langchain.tools import Tool

# Example tool: Weather API simulator
def get_weather(location: str):
    return f"The current weather in {location} is sunny and 25°C."

# Define the weather tool
weather_tool = Tool(
    name="Weather Tool",
    func=get_weather,
    description="Provides weather information for a given location."
)


In [62]:
from langchain_ollama import ChatOllama
from langchain.chains import RetrievalQA

# Initialize the Ollama LLM
llm = ChatOllama(model="mistral-nemo:latest")
print("💬 Ollama LLM loaded.")

# Create the Retrieval QA chain
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)
print("🔗 RetrievalQA chain created.")


💬 Ollama LLM loaded.
🔗 RetrievalQA chain created.


In [63]:
from langchain.agents import initialize_agent, Tool, AgentType

# Tool: Connect the document retriever to the agent
retrieval_tool = Tool(
    name="Document Retrieval",
    func=lambda q: retrieval_qa_chain.invoke(q)["result"],
    description="Retrieves answers from the document database."
)

# Combine tools into the agent
tools = [retrieval_tool, weather_tool]

# Initialize the agent with Ollama and tools
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)
print("🤖 Agent initialized with tools.")


🤖 Agent initialized with tools.


In [68]:
agent.run("WDoes Michael Johnson have any anxiety problem and what is the temperature in India now?")  # Query the agent
#agent.run("What is the weather in New York?")  # Query the weather tool



> Entering new AgentExecutor chain...
First, let's find out if there's information about Michael Johnson having any anxiety problems. Then we'll check the current temperature in India.

Action: Document Retrieval
Action Input: "Michael Johnson anxiety"
Observation: Based on the provided context, Michael Johnson is experiencing sudden episodes of intense chest pain, heart palpitations, shortness of breath, dizziness, sweating, and a feeling of impending doom. These symptoms suggest that he might be suffering from panic attacks, which are often associated with anxiety disorders.

Here's why:

1. **Symptoms**: The symptoms Michael is experiencing are classic symptoms of panic attacks.
2. **Anxiety History**: He reports using Lorazepam (a medication often used to treat anxiety) as needed for anxiety.
3. **Lifestyle Factors**: His high-stress job as an investment banker could be a contributing factor.

However, I don't know if Michael has been formally diagnosed with panic disorder or any

"Michael Johnson might be experiencing anxiety problems as indicated by his symptoms, though it's not confirmed without a formal diagnosis. Currently, the temperature in India is 25°C with sunny weather."